In [34]:
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle

<h2> EXTRACTING FIFA 2022 GROUP TABLES </h2>

In [2]:
tables= pd.read_html("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup")

In [36]:
tables[12] # 8 tables
tables[19]
tables[26]
tables[61]

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Portugal,0,0,0,0,0,0,0,0
1,2,Ghana,0,0,0,0,0,0,0,0
2,3,Uruguay,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


In [15]:
tables[12].columns

Index(['Pos',
       'Team.mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:"[ "}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:" ]"}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vte',
       'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Qualification'],
      dtype='object')

In [457]:
dict_table={}
for letter,i in zip(alphabet,range (12,62,7)):
    df_table=tables[i]
    df_table.rename(columns={df_table.columns[1]: "Team"}, inplace=True)
    df_table.pop("Qualification")
    dict_table[f"Group {letter}"]=df_table
    
    

In [461]:
dict_table["Group G"]

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Brazil,0,0,0,0,0,0,0,0
1,2,Serbia,0,0,0,0,0,0,0,0
2,3,Switzerland,0,0,0,0,0,0,0,0
3,4,Cameroon,0,0,0,0,0,0,0,0


In [35]:
with open("dict_table","wb") as output:
    pickle.dump(dict_table,output)

In [38]:
years=[]
for i  in range(1930,2019,4):
    years.append(i)

In [43]:
years

[1930,
 1934,
 1938,
 1950,
 1954,
 1958,
 1962,
 1966,
 1970,
 1974,
 1978,
 1982,
 1986,
 1990,
 1994,
 1998,
 2002,
 2006,
 2010,
 2014,
 2018]

<h1> WEB SCRAPING MATCH RESULTS FROM 1930-2018</h1>

In [45]:
from bs4 import BeautifulSoup
import requests

In [142]:
def get_matches(year):
    web=f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, "lxml")

    matches=soup.find_all("div",class_="footballbox")

    home_teams = []
    score = []
    away_teams = []

    for match in matches:
        home_team = match.find("th", class_="fhome").get_text()
        match_score = match.find("th", class_="fscore").get_text()
        away_team = match.find("th", class_="faway").get_text()

        home_teams.append(home_team)
        score.append(match_score)
        away_teams.append(away_team)

    football = {'Home': home_teams , 'Score': score , 'Away': away_teams}
    df_football = pd.DataFrame(football)
    df_football['Year'] = year
    return df_football
    
fifa = [get_matches(year) for year in years] 

df_f = pd.concat(fifa, ignore_index = True)


<h2>GETTING MISSING MATCH DATA FOR YEAR 1990 </h2>

In [143]:
def get_missing_matches(year):
    web="https://en.wikipedia.org/wiki/1990_FIFA_World_Cup"
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, "lxml")

    matches=soup.find_all("tr", attrs={"style":"font-size:90%"})

    home_teams = []
    score = []
    away_teams = []


    for match in matches:
        home_team = match.find("a").get_text()
        match_score = match.find("b").get_text()
        away_team = match.find("span", attrs={"style":"white-space:nowrap"}).get_text()


        home_teams.append(home_team)
        score.append(match_score)
        away_teams.append(away_team)

    football = {'Home': home_teams, 'Score': score , 'Away': away_teams}
    df_football = pd.DataFrame(football)
    df_football['Year'] = year
    return df_football

missing_fifa_data = get_missing_matches(1990)

<h1>WEB SCRAPING MATCH FIXTURES FOR FIFA 2022 </h1>

In [156]:
def get_fixtures(year):
    web = "https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup"
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, "lxml")

    matches=soup.find_all("div",class_="footballbox")

    home_22 = []
    score_2022 = []
    away_22 = []

    for match in matches:
        home_team_2022 = match.find("th", class_="fhome").get_text()
        match_score_2022 = match.find("th", class_="fscore").get_text()
        away_team_2022 = match.find("th", class_="faway").get_text()

        home_22.append(home_team_2022)
        score_2022.append(match_score_2022)
        away_22.append(away_team_2022)

    fixtures = {'Home': home_22 , 'Score': score_2022 , 'Away': away_22}
    df_fixtures = pd.DataFrame(fixtures)
    return df_fixtures

df_fixtures_2022 = get_fixtures(2022)
df_fixtures_2022.to_csv("FIFA 2022 FIXTURES.csv", index = False)

In [149]:
df_fifa = pd.concat([df_f, missing_fifa_data], ignore_index = True)  # To concatenate 2 dataframes pass them as a list
df_fifa.to_csv('FIFA HISTORICAL DATA.csv', index = False)

<h1> CLEANING DATA FOR ANALYSIS </h1>

In [165]:
# CLEANING DATA REMOVING SPACES

df_fixtures_2022['Home'] = df_fixtures_2022['Home'].str.strip()
df_fixtures_2022['Away'] = df_fixtures_2022['Away'].str.strip()

# SORTING THE FIFA DATA VALUES ACCORDING TO YEAR

df_fifa.sort_values('Year', inplace = True)

In [279]:
# REPLACING STRINGS THAT CONTAIN "After extra time"

df_fifa['Score'] = df_fifa['Score'].str.replace('(a.e.t)','', regex = True)



In [298]:
# DROPPING THE WALKOVER MATCH BETWEEN SWEDEN AND AUSTRIA IN 1938

df_fifa[df_fifa['Score'].str.contains('a')]
df_fifa.drop(index=37, inplace= True)


,Home,Score,Away,Year


In [299]:
# CLEANING WHITE SPACES FOR FIFA DATASET

df_fifa['Home'] = df_fifa['Home'].str.strip()
df_fifa['Away'] = df_fifa['Away'].str.strip()

In [310]:
# SEPERATING SCORE INTO HOME AND AWAY GOALS

df_fifa[['Home Goals', 'Away Goals']] = df_fifa['Score'].str.split('–', expand = True)
df_fifa = df_fifa.drop('Score', axis=1)

In [326]:
df_fifa.rename(columns = {'Home':'Home Team', 'Away': 'Away Teams'}, inplace= True)

,Home Team,Away Teams,Year,Home Goals,Away Goals
0,France,Mexico,1930,4,1
17,Uruguay,Argentina,1930,4,2
16,Uruguay,Yugoslavia,1930,6,1
15,Argentina,United States,1930,6,1
14,Paraguay,Belgium,1930,1,0
...,...,...,...,...,...
829,Serbia,Brazil,2018,0,2
828,Serbia,Switzerland,2018,1,2
827,Brazil,Costa Rica,2018,2,0
825,Costa Rica,Serbia,2018,0,1


In [342]:
df_fifa.dtypes

Home Team     object
Away Teams    object
Year           int64
Home Goals    object
Away Goals    object
dtype: object

In [349]:
df_fifa['Total Goals'] = df_fifa['Home Goals'] + df_fifa['Away Goals']

In [354]:
df_fifa = df_fifa.drop("Total Goals", axis = 1)

In [355]:
df_fifa

,Home Team,Away Teams,Year,Home Goals,Away Goals
0,France,Mexico,1930,4,1
17,Uruguay,Argentina,1930,4,2
16,Uruguay,Yugoslavia,1930,6,1
15,Argentina,United States,1930,6,1
14,Paraguay,Belgium,1930,1,0
...,...,...,...,...,...
829,Serbia,Brazil,2018,0,2
828,Serbia,Switzerland,2018,1,2
827,Brazil,Costa Rica,2018,2,0
825,Costa Rica,Serbia,2018,0,1


In [ ]:
# REMOVING UNWANTED STRINGS FROM THE DATASET FOR CONVERSION

df_fifa[df_fifa['Away Goals'].str.contains("0(.)")]
df_fifa['Away Goals'] = df_fifa['Away Goals'].str.replace('0(.)','0', regex = True)

In [451]:
# CHANGING THE DATA TYPE OF COLUMNS

df_fifa['Home Goals'] = pd.to_numeric(df_fifa['Home Goals'], downcast = 'integer', errors='raise')
df_fifa['Away Goals'] = pd.to_numeric(df_fifa['Away Goals'], downcast = 'integer', errors='raise')

In [453]:
# CREATING TOTAL GOALS COLUMN

df_fifa['Total Goals'] = df_fifa['Home Goals'] + df_fifa['Away Goals']

In [454]:
df_fifa

,Home Team,Away Teams,Year,Home Goals,Away Goals,Total Goals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
829,Serbia,Brazil,2018,0,2,2
828,Serbia,Switzerland,2018,1,2,3
827,Brazil,Costa Rica,2018,2,0,2
825,Costa Rica,Serbia,2018,0,1,1


In [455]:
# EXPORTING CLEAN DATA 

df_fifa.to_csv('FIFA HISTORICAL DATA CLEANED.csv', index = False)
df_fixtures_2022.to_csv("FIFA 2022 FIXTURES CLEANED.csv", index = False)